# 新版資料庫

搭啷！這個就是我們新版的資料庫，新增了
* 殖利率
* 三大法人
* 上櫃所有股票
* 更快速的存取方式

In [1]:
from finlab.crawler import (

    # 爬蟲們
    crawl_price,
    crawl_bargin,
    crawl_pe,
    crawl_monthly_report,
    crawl_finance_statement_by_date,
    crawl_twse_divide_ratio,
    crawl_otc_divide_ratio,
    crawl_twse_cap_reduction,
    crawl_otc_cap_reduction,
    crawl_benchmark,
    
    # 時間的range
    date_range,
    month_range,
    season_range,
    
    # GUI介面
    widget, out,
)

widget('price', crawl_price, date_range)
widget('bargin_report', crawl_bargin, date_range)
widget('benchmark', crawl_benchmark, date_range)
widget('pe', crawl_pe, date_range)
widget('monthly_report', crawl_monthly_report, month_range)
widget('financial_statement', crawl_finance_statement_by_date, season_range)
widget('twse_divide_ratio', crawl_twse_divide_ratio)
widget('otc_divide_ratio', crawl_otc_divide_ratio)
widget('twse_cap_reduction', crawl_twse_cap_reduction)
widget('otc_cap_reduction', crawl_otc_cap_reduction)

out.clear_output()
display(out)

Output(layout=Layout(border='1px solid black'))

In [11]:
# date_range_record_file = os.path.join('history', 'date_range.pickle')
# table_name = "price"

# def table_date_range(table_name):
#     if os.path.isfile(date_range_record_file):
#         with open(date_range_record_file, 'rb') as f:
#             dates = pickle.load(f)
#             print(dates)
#             if table_name in dates:
#                 return dates[table_name]
#             else:
#                 return [None, None]
#     else:
#         return [None, None]

# first_date, last_date = table_date_range(table_name)

# # label = widgets.Label(table_name + ' | ' + str(first_date) + ' ~ ' + str(last_date))

# print(first_date)

None


In [ ]:
import auto_update

## 1. 資料調用

In [2]:
import pandas as pd
df = pd.read_pickle("history/tables/price.pkl")
df.head()

成交股數 成交筆數     成交金額   收盤價   開盤價   最低價   最高價 最後揭示買價  \
stock_id date                                                              
0015 富邦  2007-04-23  104000   12   992070  9.54  9.45  9.45  9.55   9.54   
         2007-04-24  187000   13  1783740  9.54  9.54  9.50  9.54   9.54   
         2007-04-25   90000   20   856220  9.52  9.50  9.50  9.53   9.52   
         2007-04-26  102000   13   978560  9.59  9.53  9.53  9.60   9.59   
         2007-04-27   26000    4   248300  9.55  9.55  9.55  9.55   9.55   

                    最後揭示賣價  
stock_id date               
0015 富邦  2007-04-23   9.57  
         2007-04-24   9.57  
         2007-04-25   9.53  
         2007-04-26   9.60  
         2007-04-27   9.58

In [2]:
from finlab.crawler import commit

commit()

twse_divide_ratio -- 資料日期
twse_divide_ratio -- 股票代號
twse_divide_ratio -- 股票名稱
twse_divide_ratio -- 除權息前收盤價
twse_divide_ratio -- 除權息參考價
twse_divide_ratio -- 權值+息值
twse_divide_ratio -- 權/息
twse_divide_ratio -- 漲停價格
twse_divide_ratio -- 跌停價格
twse_divide_ratio -- 開盤競價基準
twse_divide_ratio -- 減除股利參考價
twse_divide_ratio -- 詳細資料
twse_divide_ratio -- 最近一次申報資料 季別/日期
twse_divide_ratio -- 最近一次申報每股 (單位)淨值
twse_divide_ratio -- 最近一次申報每股 (單位)盈餘
twse_divide_ratio -- twse_divide_ratio
income_sheet -- 不動產、廠房及設備減損損失
income_sheet -- 不動產、廠房及設備減損迴轉利益
income_sheet -- 不動產、廠房及設備重估價利益
income_sheet -- 不動產、廠房及設備重估價損失
income_sheet -- 不動產及設備減損損失
income_sheet -- 不動產及設備減損迴轉利益
income_sheet -- 不動產重估增值
income_sheet -- 不重分類至損益之其他項目
income_sheet -- 不重分類至損益之避險工具損益
income_sheet -- 不重分類至損益之項目
income_sheet -- 不重分類至損益之項目總額
income_sheet -- 不重分類至損益之項目總額(稅後)
income_sheet -- 不重分類至損益之項目總額（稅後）
income_sheet -- 交通費
income_sheet -- 交際費
income_sheet -- 什支
income_sheet -- 什項支出
income_sheet -- 什項購置
income_sheet -- 代理手續費收入
income_sheet -- 代理

## 2. 分解上述的功能

### 2.A 分解其中的功能 crawl_function

In [17]:
import requests
import datetime
import pandas as pd
from io import StringIO

def requests_get(*args1, **args2):
    i = 3
    while i >= 0:
        try:
            return requests.get(*args1, **args2)
        except (ConnectionError, ReadTimeout) as error:
            print(error)
            print('retry one more time after 60s', i, 'times left')
            time.sleep(60)
        i -= 1
    return pd.DataFrame()

date = datetime.date(2019,1,2)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
date_str = date.strftime('%Y%m%d')
res = requests_get('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+date_str+'&type=ALLBUT0999', headers=headers,)
# print(res.text)
header = np.where(list(map(lambda l: '證券代號' in l, res.text.split('\n')[:200])))[0][0]  # 找header行數
df = pd.read_csv(StringIO(res.text.replace('=','')), header=header-1)
df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,0050,元大台灣50,"8,532,073","4,102","636,107,101",75.85,75.90,73.95,74.05,-,1.45,74.05,12,74.10,6,0.00,NaN
1,0051,元大中型100,"8,000",7,"226,320",28.46,28.46,28.11,28.11,-,0.19,28.11,1,28.22,40,0.00,NaN
2,0052,富邦科技,"100,050",8,"4,763,605",48.36,48.36,47.60,47.60,-,0.51,47.21,5,47.61,5,0.00,NaN
3,0053,元大電子,"2,000",2,"61,060",30.53,30.53,30.53,30.53,+,0.03,29.93,2,30.21,20,0.00,NaN
4,0054,元大台商50,"3,000",2,"58,310",19.46,19.46,19.39,19.39,X,0.00,19.27,20,19.42,20,0.00,NaN


In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

def price_twe(date):
    date_str = date.strftime('%Y%m%d')
    res = requests_get('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+date_str+'&type=ALLBUT0999', headers=headers,)
    
    if res.text == '':
        print('holiday')
        return pd.DataFrame()

    header = np.where(list(map(lambda l: '證券代號' in l, res.text.split('\n')[:200])))[0][0]

    df = pd.read_csv(StringIO(res.text.replace('=','')), header=header-1)
    df = combine_index(df, '證券代號', '證券名稱')
    df = preprocess(df, date)
    return df

def price_otc(date):
    datestr = otc_date_str(date)
    link = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d='+datestr+'&s=0,asc,0'
    res = requests_get(link, headers=headers)
    df = pd.read_csv(StringIO(res.text), header=2)

    if len(df) < 30:
        print('holiday')
        return pd.DataFrame()

    df = combine_index(df, '代號', '名稱')
    df = preprocess(df, date)
    df = df[df['成交筆數'].str.replace(' ', '') != '成交筆數']
    return df

In [2]:
import datetime
df = crawl_price(datetime.date(2019,1,2))
df.head()

/home/rossleecooloh/.pyenv/versions/3.6.6/lib/python3.6/site-packages/pandas/core/frame.py:6701: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,,成交筆數,成交股數,成交金額,收盤價,最低價,最後揭示買價,最後揭示賣價,最高價,開盤價
stock_id,date,,,,,,,,,
0050 元大台灣50,2019-01-02,4102,8532073,636107101,74.05,73.95,74.05,74.10,75.90,75.85
0051 元大中型100,2019-01-02,7,8000,226320,28.11,28.11,28.11,28.22,28.46,28.46
0052 富邦科技,2019-01-02,8,100050,4763605,47.60,47.60,47.21,47.61,48.36,48.36
0053 元大電子,2019-01-02,2,2000,61060,30.53,30.53,29.93,30.21,30.53,30.53
0054 元大台商50,2019-01-02,2,3000,58310,19.39,19.39,19.27,19.42,19.46,19.46


### 2.B 分解其中功能 time_range

In [3]:
# 用dateutil的這個套件做出date range的功能
# from dateutil.rrule import rrule, DAILY, MONTHLY
# [dt.date() for dt in rrule(3, dtstart=datetime.date(2019, 1, 2), until=datetime.date(2019, 1, 4))]
season_range(datetime.date(2018,1,2), datetime.date(2019,1,10))

[datetime.date(2018, 3, 31),
 datetime.date(2018, 5, 15),
 datetime.date(2018, 8, 14),
 datetime.date(2018, 11, 14)]

### 2.C 將功能組裝起來 widget

In [12]:
widget("price", crawl_price, date_range)

In [27]:
df = pd.read_pickle("history/tables/monthly_report.pkl")
df["當月營收"].reset_index().pivot("date", "stock_id")["當月營收"]

stock_id,1101 台泥,1102 亞泥,1103 嘉泥,1104 環球水泥,1107 建台,1108 幸福水泥,1109 信大水泥,1110 東泥,1201 味全公司,1203 味王公司,...,9945 潤泰創新,9946 三發地產,9946 金革科技,9949 琉園,9950 萬國通路,9951 皇田工業,9955 佳龍,9958 世紀鋼構,9960 邁達康,9962 有益鋼鐵
date,,,,,,,,,,,,,,,,,,,,,
2005-01-10,2746766,1094966,271258,346111,9910,386000,190917,195701,669370,160103,...,729712,NaN,36896,33813,144783,99732,86217,NaN,37254,NaN
2005-02-10,1566092,663833,156962,203172,8378,185213,94491,124268,547472,124124,...,729620,NaN,31515,24585,115451,52500,88619,NaN,35983,NaN
2005-03-10,2504657,968271,234870,295127,21295,371569,182687,189452,658594,150567,...,745498,NaN,33723,22733,130065,125257,120604,NaN,39668,NaN
2005-04-10,2418388,876647,234952,301353,11769,372656,184408,178292,779660,134525,...,498365,NaN,27414,25029,127739,86343,104614,NaN,42114,NaN
2005-05-10,2394229,934672,221134,309846,11894,341375,176447,169457,874858,134959,...,579833,NaN,28982,26062,155837,78049,106986,NaN,40487,NaN
2005-06-10,2276706,838304,242022,218786,5604,339798,184114,133491,933508,143116,...,576091,NaN,21128,31137,151615,77335,134200,NaN,37708,255692
2005-07-10,2302439,825931,211776,288601,7175,350093,166573,187692,1050451,158857,...,441891,NaN,24597,31653,153242,69530,102765,NaN,40012,259458
2005-08-10,2241354,854126,220881,259319,8819,360394,167166,182572,987339,181231,...,695772,NaN,40837,23872,151585,86560,143874,NaN,45444,335005
2005-09-10,2379415,858750,224785,276050,8365,339455,165241,187849,920330,147064,...,545072,NaN,28127,21495,140153,97299,142127,NaN,40680,334344


## 3. 做一些小修改，方便我們之後做財經資料研究

In [5]:
# 對每一個table做一次pivot資料整理 
from finlab.crawler import commit

commit()

balance_sheet -- 一年或一營業週期內到期應付公司債
balance_sheet -- 一年或一營業週期內到期或執行賣回權公司債
balance_sheet -- 一年或一營業週期內到期長期借款
balance_sheet -- 一年或一營業週期內到期長期應付票據及款項－關係人
balance_sheet -- 一年或一營業週期內到期長期應付票據及款項－非關係人
balance_sheet -- 一年或一營業週期內到期長期負債
balance_sheet -- 一年或一營業週期內到期長期負債合計
balance_sheet -- 上期損益
balance_sheet -- 不動產、廠房及設備
balance_sheet -- 不動產、廠房及設備合計
balance_sheet -- 不動產、廠房及設備淨額
balance_sheet -- 不動產及設備
balance_sheet -- 不動產及設備合計
balance_sheet -- 不動產及設備淨額
balance_sheet -- 不動產及設備－其他成本
balance_sheet -- 不動產及設備－其他淨額
balance_sheet -- 不動產及設備－淨額
balance_sheet -- 不動產重估增值
balance_sheet -- 不動產重估增值－採用權益法之子公司
balance_sheet -- 不動產重估增值－採用權益法之關聯企業及合資
balance_sheet -- 不動產重估增值－母公司
balance_sheet -- 中期擔保放款
balance_sheet -- 中期放款
balance_sheet -- 中華郵政轉存款
balance_sheet -- 交割代價
balance_sheet -- 交割結算基金
balance_sheet -- 交割結算基金合計
balance_sheet -- 交割結算基金－期貨
balance_sheet -- 交割結算基金－櫃檯
balance_sheet -- 交割結算基金－集中
balance_sheet -- 交通及運輸設備
balance_sheet -- 交通及運輸設備淨額
balance_sheet -- 人壽保險現金解約價值
balance_sheet -- 什項設備
balance_sheet -- 代付款

MemoryError: 

## 4. 將做完修改的資料拿出來

In [6]:
df = pd.read_pickle("history/items/price/收盤價.pkl")
df.head()

stock_id,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2007-04-23,9.54,57.85,32.83,38.40,NaN,NaN,NaN,NaN,NaN,NaN,...,31.0,30.35,9.78,73.6,13.25,51.3,72.4,NaN,46.0,49.6
2007-04-24,9.54,58.10,32.99,38.65,NaN,NaN,NaN,NaN,NaN,NaN,...,32.0,30.50,9.91,75.0,13.25,50.5,71.6,NaN,45.9,50.4
2007-04-25,9.52,57.60,32.80,38.59,NaN,NaN,NaN,NaN,NaN,NaN,...,32.3,29.50,10.10,74.6,13.30,49.9,71.6,NaN,49.1,49.1
2007-04-26,9.59,57.70,32.80,38.60,NaN,NaN,NaN,NaN,NaN,NaN,...,31.6,29.15,10.80,74.5,13.25,49.5,71.0,NaN,48.9,48.0
2007-04-27,9.55,57.50,32.72,38.40,NaN,NaN,NaN,NaN,NaN,NaN,...,31.4,28.50,11.55,75.7,13.15,48.8,69.5,NaN,48.6,46.5


# AUTO update 每日自動執行更新

有時候你可能會希望將所有的程式

每天自動執行就好，不然還要開jupyter -> 啟動notebook -> 執行GUI -> 點擊好幾次GUI

才能完成更新的動作
.
接下來我們就來教怎麼樣直接一次更新吧！

In [10]:
import inspect
import datetime

sig = inspect.signature(crawl_price)
print(sig.parameters)
print(len(sig.parameters))
if date_range(datetime.date(2019, 7, 21), datetime.date(2019, 7, 21)):
    print(123)

OrderedDict([('date', <Parameter "date">)])
1
123


In [2]:
from finlab.crawler import table_date_range, update_table, to_pickle, commit
import datetime
import inspect


table_name = 'price'
crawl_function = crawl_twse_cap_reduction

def auto_update(table_name, crawl_function, time_range=None):
    
    sig = inspect.signature(crawl_function)
    
    # 先檢查是否需要date參數
    if len(sig.parameters) != 0:
        
        start_date = table_date_range(table_name)[1]
        end_date = datetime.datetime.now() 

        dates = time_range(start_date, end_date)
        # 避免date輸入錯誤時沒日期
        if dates:
            update_table(table_name, crawl_function, dates)

    else:
        df = crawl_function()
        to_pickle(df, table_name)
        

auto_update('price', crawl_price, date_range)
auto_update('bargin_report', crawl_bargin, date_range)
auto_update('pe', crawl_pe, date_range)
auto_update('monthly_report', crawl_monthly_report, month_range)
auto_update('financial_statement', crawl_finance_statement_by_date, season_range)
auto_update('twse_divide_ratio', crawl_twse_divide_ratio)
auto_update('otc_divide_ratio', crawl_otc_divide_ratio)
auto_update('twse_cap_reduction', crawl_twse_cap_reduction)
auto_update('otc_cap_reduction', crawl_otc_cap_reduction)

commit()  # 轉換資料格式

monthly_report -- 當月營收
monthly_report -- 上月營收
monthly_report -- 去年當月營收
monthly_report -- 上月比較增減(%)
monthly_report -- 去年同月增減(%)
monthly_report -- 當月累計營收
monthly_report -- 去年累計營收
monthly_report -- 前期比較增減(%)
income_sheet_cumulate -- 累計不動產、廠房及設備減損損失
income_sheet_cumulate -- 累計不動產、廠房及設備減損迴轉利益
income_sheet_cumulate -- 累計不動產、廠房及設備重估價利益
income_sheet_cumulate -- 累計不動產、廠房及設備重估價損失
income_sheet_cumulate -- 累計不動產及設備減損損失
income_sheet_cumulate -- 累計不動產及設備減損迴轉利益
income_sheet_cumulate -- 累計不動產重估增值
income_sheet_cumulate -- 累計不重分類至損益之其他項目
income_sheet_cumulate -- 累計不重分類至損益之避險工具損益
income_sheet_cumulate -- 累計不重分類至損益之項目
income_sheet_cumulate -- 累計不重分類至損益之項目總額
income_sheet_cumulate -- 累計不重分類至損益之項目總額(稅後)
income_sheet_cumulate -- 累計不重分類至損益之項目總額稅後
income_sheet_cumulate -- 累計不重分類至損益之項目總額（稅後）
income_sheet_cumulate -- 累計交通費
income_sheet_cumulate -- 累計交際費
income_sheet_cumulate -- 累計什支
income_sheet_cumulate -- 累計什項支出
income_sheet_cumulate -- 累計什項購置
income_sheet_cumulate -- 累計代理手續費收入
income_sheet_cumulate -- 累計代理業收入（

income_sheet_cumulate -- 累計其他費損
income_sheet_cumulate -- 累計其他費用
income_sheet_cumulate -- 累計其他資產減損損失
income_sheet_cumulate -- 累計其他資產減損迴轉利益
income_sheet_cumulate -- 累計其他金融資產減損損失
income_sheet_cumulate -- 累計其他金融資產減損迴轉利益
income_sheet_cumulate -- 累計其他長期員工福利
income_sheet_cumulate -- 累計其他餐旅服務成本
income_sheet_cumulate -- 累計其他餐旅服務收入
income_sheet_cumulate -- 累計具金融商品性質之保險契約準備淨變動
income_sheet_cumulate -- 累計再保佣金支出
income_sheet_cumulate -- 累計再保佣金收入
income_sheet_cumulate -- 累計再保手續費支出
income_sheet_cumulate -- 累計再保手續費收入
income_sheet_cumulate -- 累計再保費支出
income_sheet_cumulate -- 累計再保費收入
income_sheet_cumulate -- 累計再保賠款與給付
income_sheet_cumulate -- 累計再保險佣金支出
income_sheet_cumulate -- 累計出售不良債權損失
income_sheet_cumulate -- 累計出售不良債權收入
income_sheet_cumulate -- 累計出售不良債權淨利益（損失）
income_sheet_cumulate -- 累計出售不良債權淨損益
income_sheet_cumulate -- 累計出售不良資產損失
income_sheet_cumulate -- 累計出售承受擔保品利益
income_sheet_cumulate -- 累計出售承受擔保品損失
income_sheet_cumulate -- 累計出售承受擔保品淨利益（損失）
income_sheet_cumulate -- 累計出售承受擔保品淨損益
income_sheet_cumu

income_sheet_cumulate -- 累計技術服務收入折讓
income_sheet_cumulate -- 累計技術服務收入淨額
income_sheet_cumulate -- 累計技術服務費用
income_sheet_cumulate -- 累計投資之預期信用減損損失及迴轉利益
income_sheet_cumulate -- 累計投資之預期信用減損迴轉利益
income_sheet_cumulate -- 累計投資型保險商品佣金(分潤)收入
income_sheet_cumulate -- 累計投資型保險商品佣金分潤收入
income_sheet_cumulate -- 累計投資性不動產利益
income_sheet_cumulate -- 累計投資性不動產損失
income_sheet_cumulate -- 累計投資性不動產損益
income_sheet_cumulate -- 累計投資性不動產減損損失
income_sheet_cumulate -- 累計投資性不動產減損迴轉利益
income_sheet_cumulate -- 累計投資性不動產費用及損失
income_sheet_cumulate -- 累計投資損失
income_sheet_cumulate -- 累計投資損失合計
income_sheet_cumulate -- 累計投資損益
income_sheet_cumulate -- 累計投資支出（投資公司適用）合計
income_sheet_cumulate -- 累計投資收入
income_sheet_cumulate -- 累計投資收入合計
income_sheet_cumulate -- 累計投資收入（投資公司適用）合計
income_sheet_cumulate -- 累計投資有價證券利息收入（持有票債券利息收入）
income_sheet_cumulate -- 累計投資減損損失
income_sheet_cumulate -- 累計投資減損損失及迴轉利益
income_sheet_cumulate -- 累計投資減損損失及迴轉利益淨額
income_sheet_cumulate -- 累計投資減損迴轉利益
income_sheet_cumulate -- 累計投資（交易）人保護費用
income_sheet_c

income_sheet_cumulate -- 累計收回外匯價格變動準備
income_sheet_cumulate -- 累計收回未決賠款準備
income_sheet_cumulate -- 累計收回未滿期保費準備
income_sheet_cumulate -- 累計收回未滿期保費準備－分入再保
income_sheet_cumulate -- 累計收回未滿期保費準備－分出再保
income_sheet_cumulate -- 累計收回未滿期保費準備－直接承保
income_sheet_cumulate -- 累計收回特別準備
income_sheet_cumulate -- 累計收回負債適足準備
income_sheet_cumulate -- 累計收回負債適足準備－分入再保
income_sheet_cumulate -- 累計收回負債適足準備－分出再保
income_sheet_cumulate -- 累計收回負債適足準備－直接承保
income_sheet_cumulate -- 累計收回責任準備
income_sheet_cumulate -- 累計收回賠款準備
income_sheet_cumulate -- 累計收回賠款準備－分入再保
income_sheet_cumulate -- 累計收回賠款準備－分出再保
income_sheet_cumulate -- 累計收回賠款準備－直接承保
income_sheet_cumulate -- 累計收回輸出保險準備
income_sheet_cumulate -- 累計收益
income_sheet_cumulate -- 累計收益分配金（信託投資公司適用）
income_sheet_cumulate -- 累計收益合計
income_sheet_cumulate -- 累計放款呆帳費用
income_sheet_cumulate -- 累計放款呆帳費用調整
income_sheet_cumulate -- 累計放款手續費收入
income_sheet_cumulate -- 累計放款手續費收入(企金)
income_sheet_cumulate -- 累計放款手續費收入(消金)
income_sheet_cumulate -- 累計放款手續費收入企金
income_sheet_cumulate --

income_sheet_cumulate -- 累計繼續營業單位本期稅後淨利（淨損）
income_sheet_cumulate -- 累計繼續營業單位本期純益（純損）
income_sheet_cumulate -- 累計繼續營業單位淨利（損）
income_sheet_cumulate -- 累計繼續營業單位淨利（淨損）
income_sheet_cumulate -- 累計繼續營業單位稅前損益
income_sheet_cumulate -- 累計繼續營業單位稅前淨利（淨損）
income_sheet_cumulate -- 累計繼續營業單位稅前純益（純損）
income_sheet_cumulate -- 累計繼續營業單位稅後淨利（淨損）
income_sheet_cumulate -- 累計職工福利
income_sheet_cumulate -- 累計股利收入
income_sheet_cumulate -- 累計股利收入合計
income_sheet_cumulate -- 累計股務代理收入
income_sheet_cumulate -- 累計股務代理收入合計
income_sheet_cumulate -- 累計育樂成本
income_sheet_cumulate -- 累計育樂成本合計
income_sheet_cumulate -- 累計育樂收入
income_sheet_cumulate -- 累計育樂收入折讓
income_sheet_cumulate -- 累計育樂收入淨額
income_sheet_cumulate -- 累計自營經手費支出
income_sheet_cumulate -- 累計自營經手費支出合計
income_sheet_cumulate -- 累計自營經手費支出－期貨
income_sheet_cumulate -- 累計自營經手費支出－櫃檯
income_sheet_cumulate -- 累計自營經手費支出－集中
income_sheet_cumulate -- 累計自由捐贈
income_sheet_cumulate -- 累計自留保險賠款與給付
income_sheet_cumulate -- 累計自留滿期保費收入
income_sheet_cumulate -- 累計與不重分類之項目相關之所得稅
inco

...

balance_sheet -- 待分配股票股利
balance_sheet -- 待分配股票股利（增資準備）
balance_sheet -- 待售房地
balance_sheet -- 待售車位
balance_sheet -- 待抵銷報失資產
balance_sheet -- 待抵銷大陸及國外負債
balance_sheet -- 待抵銷大陸及國外資產
balance_sheet -- 待抵銷大陸地區及國外負債
balance_sheet -- 待抵銷大陸地區及國外資產
balance_sheet -- 待處分股權投資
balance_sheet -- 待處分股權投資淨額
balance_sheet -- 待註銷股本
balance_sheet -- 待註銷股本股數
balance_sheet -- 待註銷股本股數（單位：股）
balance_sheet -- 待退回產品權利－流動
balance_sheet -- 待退回產品權利－非流動
balance_sheet -- 意外損失準備
balance_sheet -- 應付交割帳款
balance_sheet -- 應付交割帳款－受託買賣
balance_sheet -- 應付交割帳款－非受託買賣
balance_sheet -- 應付代售印花稅票
balance_sheet -- 應付代收保險費
balance_sheet -- 應付代收款
balance_sheet -- 應付佣金
balance_sheet -- 應付保管款
balance_sheet -- 應付保險賠款與給付
balance_sheet -- 應付借券－避險
balance_sheet -- 應付借券－避險合計
balance_sheet -- 應付借券－避險評價調整
balance_sheet -- 應付借券－避險－股票
balance_sheet -- 應付借券－非避險
balance_sheet -- 應付借券－非避險合計
balance_sheet -- 應付借券－非避險評價調整
balance_sheet -- 應付債券
balance_sheet -- 應付債券折價
balance_sheet -- 應付債券溢價
balance_sheet -- 應付公司債
balance_sheet -- 應付公司債合計
balance

balance_sheet -- 未完工程
balance_sheet -- 未完工程及待驗設備
balance_sheet -- 未完工程及預付房地設備款
balance_sheet -- 未實現利息收入－應收分期帳款
balance_sheet -- 未實現利息收入－應收關係人分期帳款
balance_sheet -- 未實現利息收入－長期應收分期帳款
balance_sheet -- 未實現利息收入－長期應收關係人分期帳款
balance_sheet -- 未實現售後租回利益
balance_sheet -- 未實現售後租回損失
balance_sheet -- 未實現重估增值合計
balance_sheet -- 未收代收款（海陸運輸業適用）
balance_sheet -- 未決賠款準備
balance_sheet -- 未滿期保費準備
balance_sheet -- 本期所得稅負債
balance_sheet -- 本期所得稅負債合計
balance_sheet -- 本期所得稅資產
balance_sheet -- 本期所得稅資產合計
balance_sheet -- 本期損益
balance_sheet -- 本行支票
balance_sheet -- 材料－庫存材料
balance_sheet -- 槓桿保證金契約交易交易人權益
balance_sheet -- 槓桿保證金契約交易客戶保證金專戶
balance_sheet -- 模具設備成本
balance_sheet -- 模具設備淨額
balance_sheet -- 模具設備－重估增（減）值
balance_sheet -- 機具設備成本
balance_sheet -- 機具設備淨額
balance_sheet -- 機具設備－重估增（減）值
balance_sheet -- 機器設備成本
balance_sheet -- 機器設備淨額
balance_sheet -- 機器設備－重估增（減）值
balance_sheet -- 機械及電腦設備
balance_sheet -- 權益
balance_sheet -- 權益總計
balance_sheet -- 權益總額
balance_sheet -- 歸屬於母公司業主之權益合計
balance_sheet -- 母公司暨子公司所持有之

balance_sheet -- 累計減損－備供出售金融資產－非流動
balance_sheet -- 累計減損－其他什項金融資產
balance_sheet -- 累計減損－其他待出售資產
balance_sheet -- 累計減損－其他待整理資產
balance_sheet -- 累計減損－其他無形資產
balance_sheet -- 累計減損－其他設備
balance_sheet -- 累計減損－其他資產－其他
balance_sheet -- 累計減損－其他金融資產
balance_sheet -- 累計減損－其他長期投資
balance_sheet -- 累計減損－其他雜項資產
balance_sheet -- 累計減損－其他非營業資產
balance_sheet -- 累計減損－冷凍設備
balance_sheet -- 累計減損－出租資產
balance_sheet -- 累計減損－出租資產－其他
balance_sheet -- 累計減損－出租資產－機器設備
balance_sheet -- 累計減損－分出保費不足準備
balance_sheet -- 累計減損－分出未滿期保費準備
balance_sheet -- 累計減損－分出負債適足準備
balance_sheet -- 累計減損－分出責任準備
balance_sheet -- 累計減損－分出賠款準備
balance_sheet -- 累計減損－取得合約之增額成本－流動
balance_sheet -- 累計減損－取得合約之增額成本－非流動
balance_sheet -- 累計減損－售氣及輸氣設備
balance_sheet -- 累計減損－商標權
balance_sheet -- 累計減損－土地
balance_sheet -- 累計減損－土地改良物
balance_sheet -- 累計減損－天然資源
balance_sheet -- 累計減損－專利權
balance_sheet -- 累計減損－專門技術
balance_sheet -- 累計減損－履行合約成本－流動
balance_sheet -- 累計減損－履行合約成本－非流動
balance_sheet -- 累計減損－建造中之投資性不動產
balance_sheet -- 累計減損－待出售資產
balance_sheet -- 

balance_sheet -- 透過其他綜合損益按公允價值衡量之債務工具投資未實現評價損益－母公司
balance_sheet -- 透過其他綜合損益按公允價值衡量之債務工具投資評價調整－流動
balance_sheet -- 透過其他綜合損益按公允價值衡量之債務工具投資評價調整－非流動
balance_sheet -- 透過其他綜合損益按公允價值衡量之債務工具投資－流動
balance_sheet -- 透過其他綜合損益按公允價值衡量之債務工具投資－非流動
balance_sheet -- 透過其他綜合損益按公允價值衡量之債務工具損益
balance_sheet -- 透過其他綜合損益按公允價值衡量之債務工具評價損益
balance_sheet -- 透過其他綜合損益按公允價值衡量之債務工具評價調整
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具前期累積處分損益
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資利益（損失）合計
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資利益（損失）－採用權益法之子公司
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資利益（損失）－採用權益法之關聯企業及合資
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資利益（損失）－母公司
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資未實現損益－採用權益法之子公司
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資未實現損益－採用權益法之關聯企業及合資
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資未實現損益－母公司
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資未實現評價利益損失
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資未實現評價損益－採用權益法之子公司
balance_sheet -- 透過其他綜合損益按公允價值衡量之權益工具投資未實現評價損益－採用權益法之關聯企業及合資
balance_sheet -

balance_sheet -- 預收利息
balance_sheet -- 預收房地款
balance_sheet -- 預收收入
balance_sheet -- 預收收入／遞延收入
balance_sheet -- 預收收益
balance_sheet -- 預收款項
balance_sheet -- 預收款項合計
balance_sheet -- 預收租金
balance_sheet -- 預收股本
balance_sheet -- 預收股款（權益項下）之約當發行股數
balance_sheet -- 預收股款（權益項下）之約當發行股數（單位：股）
balance_sheet -- 預收股款（股東權益項下）之約當發行股數（單位：股）
balance_sheet -- 預收訂金
balance_sheet -- 預收貨款
balance_sheet -- 飛航設備成本
balance_sheet -- 飛航設備淨額
balance_sheet -- 飛航設備－重估增（減）值
balance_sheet -- 餐飲用品（餐飲業）
price -- 成交股數
price -- 成交筆數
price -- 成交金額
price -- 收盤價
price -- 開盤價
price -- 最低價
price -- 最高價
price -- 最後揭示買價
price -- 最後揭示賣價
bargin_report -- 外陸資買進股數(不含外資自營商)
bargin_report -- 外陸資賣出股數(不含外資自營商)
bargin_report -- 外陸資買賣超股數(不含外資自營商)
bargin_report -- 外資自營商買進股數
bargin_report -- 外資自營商賣出股數
bargin_report -- 外資自營商買賣超股數
bargin_report -- 投信買進股數
bargin_report -- 投信賣出股數
bargin_report -- 投信買賣超股數
bargin_report -- 自營商買進股數(自行買賣)
bargin_report -- 自營商賣出股數(自行買賣)
bargin_report -- 自營商買賣超股數(自行買賣)
bargin_report -- 自營商買進股數(避險)
bargin_report -- 自營

## 加入新的benchmark data

In [10]:
# 時間物件
import datetime
import requests
import pandas as pd
from io import StringIO


# 下載某天的資料
date = datetime.date(2018,1,3)

def crawl_benchmark(date):
    date_str = date.strftime('%Y%m%d')
    res = requests.get("http://www.twse.com.tw/exchangeReport/MI_5MINS_INDEX?response=csv&date=" + 
                       date_str + "&_=1544020420045")
    
    if len(res.text) < 100:
        print('holiday')
        return pd.DataFrame()

    # 利用 pandas 將資料整理成表格
    df = pd.read_csv(StringIO(res.text.replace("=","")), header=1, index_col='時間')

    # 資料處理 (整理成存成pkl檔的格式)
    df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df.index = pd.to_datetime(date.strftime('%Y %m %d ') + pd.Series(df.index))
    df = df.apply(lambda s: s.astype(str).str.replace(",", "").astype(float))

    df.head()

    df.index.name = 'date'
    df = df.reset_index()
    df['stock_id'] = '台股指數'
    df = df.set_index(['stock_id', 'date'])
    return df

from finlab.crawler import widget, date_range, out

widget('benchmark', crawl_benchmark, date_range)

out.clear_output()
display(out)

Output(layout=Layout(border='1px solid black'), outputs=({'name': 'stdout', 'text': 'start crawl benchmark fro…

## 查看存放好的benchmark data

In [12]:
from finlab.crawler import commit

commit()

twse_divide_ratio -- 資料日期
twse_divide_ratio -- 股票代號
twse_divide_ratio -- 股票名稱
twse_divide_ratio -- 除權息前收盤價
twse_divide_ratio -- 除權息參考價
twse_divide_ratio -- 權值+息值
twse_divide_ratio -- 權/息
twse_divide_ratio -- 漲停價格
twse_divide_ratio -- 跌停價格
twse_divide_ratio -- 開盤競價基準
twse_divide_ratio -- 減除股利參考價
twse_divide_ratio -- 詳細資料
twse_divide_ratio -- 最近一次申報資料 季別/日期
twse_divide_ratio -- 最近一次申報每股 (單位)淨值
twse_divide_ratio -- 最近一次申報每股 (單位)盈餘
twse_divide_ratio -- twse_divide_ratio
income_sheet -- 不動產、廠房及設備減損損失
income_sheet -- 不動產、廠房及設備減損迴轉利益
income_sheet -- 不動產、廠房及設備重估價利益
income_sheet -- 不動產、廠房及設備重估價損失
income_sheet -- 不動產及設備減損損失
income_sheet -- 不動產及設備減損迴轉利益
income_sheet -- 不動產重估增值
income_sheet -- 不重分類至損益之其他項目
income_sheet -- 不重分類至損益之避險工具損益
income_sheet -- 不重分類至損益之項目
income_sheet -- 不重分類至損益之項目總額
income_sheet -- 不重分類至損益之項目總額(稅後)
income_sheet -- 不重分類至損益之項目總額（稅後）
income_sheet -- 交通費
income_sheet -- 交際費
income_sheet -- 什支
income_sheet -- 什項支出
income_sheet -- 什項購置
income_sheet -- 代理手續費收入
income_sheet -- 代理

In [8]:
from finlab.data import Data

data = Data()

df = data.get("發行量加權股價指數")
df.head()

stock_id,台股指數
date,
2006-01-02 09:00:00,6548.34
2006-01-02 09:01:00,6457.61
2006-01-02 09:02:00,6452.82
2006-01-02 09:03:00,6452.39
2006-01-02 09:04:00,6451.61
